Austion Project Directory Info

In [10]:
import arcpy
import os
pathname = r"c:\Users\cpjoh\OneDrive\Desktop\GIS_4057\Homeworks\geog4057_Johnson_C_HW8\ex8"
aprx = arcpy.mp.ArcGISProject(os.path.join(pathname,"Austin.aprx"))
print(aprx.defaultGeodatabase)

c:\Users\cpjoh\OneDrive\Desktop\GIS_4057\Homeworks\geog4057_Johnson_C_HW8\ex8\Austin_Data.gdb


In [39]:
aprx.saveACopy(os.path.join(pathname,'Austin_Copy.aprx'))


I cannot seem to find an answer to why the copied file is smaller than the original. All I can think of is that perhaps the new file is compressed or maybe there's some "open" data I cannot find in the original that doesn't get copied over.

In [40]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
Region
Foot_US


Opening the project file in the script locks the file from being manipulated by outside sources.  The "del" function effectively closes it.  It does not actually delete the project.

In [41]:
aprx = arcpy.mp.ArcGISProject('CURRENT') ###defines the prjoect we want to open and manipulate
m = aprx.listMaps("Region")[0]    ###[0] because it puts the function into a list, but there's only the one so we don't need to worry about [:]
m.name = "County"
#m = aprx.listMaps("County")[0]   ### In case it needs to be turned back
#m.name = "Region"
del aprx

In [42]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)    ### prints each layer under their respective maps.
del aprx

Map: Downtown
Light Gray Reference
trees
parks
base
Light Gray Base
Map: County
facilities
hospitals
parks
Topographic


In [43]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]         ###Now looking at just the downtown map
lyrs = m.listLayers()                    ### list of the layers of the map provided
for lyr in lyrs:
    if lyr.isBasemapLayer:               ###Boolean, same for feature layer
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx


Light Gray Reference is a basemap layer
trees is a feature layer
parks is a feature layer
base is a feature layer
Light Gray Base is a basemap layer


In [11]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")
#m.addbasemap adds the light gray basemap and changes it from the base topographic one. 
###I also note that we do not close the door on this file so to speak, with the del aprx.  I believe we should still be able to use the data but cannot add more

test


In [12]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]    ###specifying "parks" still puts it in a list so we use [0]
sym = lyr.symbology
#green = {"RGB": [100, 175, 0, 100]}         ###[r,g,b,alpha]
brown = {"RGB": [185, 150, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):  ###if this layer is one that has rendering capabilities
    sym.renderer.symbol.color = brown #green
    lyr.symbology = sym

The color property uses a dictionary to accept several possible modifiable color setting options.  In this case we are using the Red,Green,Blue method. The values of the dictionary are broken down by [Red, Green, Blue, Alpha(opacity)]
Brown is easily achievable through mixing Green and Red so we can mainpulate the values or use a color picking tool/googling the brown shade we want in rgb colors to set them to the brown color.

In [13]:
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')  ###creates 11x8.5 rectangle (W x H)

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]   ###with an inputed bottom left corner as input and the desired width and height it creats a polygon used below for the map frame
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])   ###to create a polygon points are needed
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")    ###creates a map frame using function above. creates a 10x7.5 inch frame on our 11x8.5 inch layout (.5 inches on all sides for a padding)

In [14]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]   ###bring in the desired compass rose
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,        ###Where you want it to be centered
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]                ###bring in the colorbar, creates it, doesn't bring it in
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)      ###using that function to create another polygon to size the scale bar
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')

In [15]:
legSi = aprx.listStyleItems('ArcGIS 2D', 'LEGEND', 'Legend 3' )[0]                             ###grab a legend design
leg = lyt.createMapSurroundElement(arcpy.Point(1,7), 'LEGEND', mf, legSi, 'New Legend Element') ###creates the legend
leg.elementWidth = 3
leg.elementHeight = 3
leg.fittingStrategy = 'AdjustFontSize'   ###parameters to adjust the look of the legend
leg.columnCount = 1
leg.title = 'Downtown'

In [16]:
lyt.exportToPDF(os.path.join(pathname, 'downtown1.pdf'))   ###exports the created map with the layout and framing as pdf
#print("done")

done


The layout is what the holds the map image/frame and is set to be 8.5 inches in height and 11 inches in width.
The map frame is what depicts the map and is created using a polygon and coordinates in reference to the layout.  So here we make the map frame 7.5 inches in height and 10 inches in width to provide a half-inch of framing on all sides.

The benefit of this metho-in my opinion-is how organized everything is.  I am relatively new to GIS and the processes to create and activate maps, creating legends, trying to manipulate the text on them is convoluted.  Here you set your parameters and you're off to the races.  You can also be very precise with what sizes you want your maps to be instead of fumbling around with the mouse to set them.  